# Semantic segmentation with LRASPP MobileNet v3 and OpenVINO

The LRASPP model is based on the [Searching for MobileNetV3](https://arxiv.org/abs/1905.02244) paper. According to the paper, Searching for MobileNetV3, LR-ASPP or Lite Reduced Atrous Spatial Pyramid Pooling has a lightweight and efficient segmentation decoder architecture. he model is pre-trained on the [MS COCO](https://cocodataset.org/#home) dataset. Instead of training on all 80 classes, the segmentation model has been trained on 20 classes from the [PASCAL VOC](http://host.robots.ox.ac.uk/pascal/VOC/) dataset:
***background*, *aeroplane*, *bicycle*, *bird*, *boat*, *bottle*, *bus*, *car*, *cat*, *chair*, *cow*, *dining table*, *dog*, *horse*, *motorbike*, *person*, *potted plant*, *sheep*, *sofa*, *train*, *tv monitor***

More information about the model is available in the [torchvision documentation](https://pytorch.org/vision/main/models/lraspp.html)

### Table of content:
- [Prerequisites](#Prerequisites)
- [Download and prepare model](#Download-and-prepare-model)
- [Convert the original model to OpenVINO IR Format](#Convert-the-original-model-to-OpenVINO-IR-Format)
- [Show Results](#Show-results)
    - [Load and preprocess an input image](#Load-and-preprocess-an-input-image)
    - [Run an inference on the OpenVINO model](#Run-an-inference-on-the-OpenVINO-model)
    - [Run an inference on the PyTorch model to compare results](#Run-an-inference-on-the-PyTorch-model-to-compare-results)

## Prerequisites

In [ ]:
%pip install -q "torch==2.0.1" "torchvision==0.15.2" datasets
%pip install -q  "openvino>=2023.1.0"

In [ ]:
from pathlib import Path

import openvino as ov
import torch
from torchvision.models.segmentation import lraspp_mobilenet_v3_large, LRASPP_MobileNet_V3_Large_Weights

# Fetch the notebook utils script from the openvino_notebooks repo
import urllib.request
urllib.request.urlretrieve(
    url='https://raw.githubusercontent.com/openvinotoolkit/openvino_notebooks/main/notebooks/utils/notebook_utils.py',
    filename='notebook_utils.py'
)
from notebook_utils import download_file

## Download and prepare model
Set a name for the model, then define width and height of the image that will be used by the network during inference. According to the input transforms function, the model is pre-trained on images with a height of 520 and width of 780.

In [ ]:
IMAGE_WIDTH = 640
IMAGE_HEIGHT = 480
DIRECTORY_NAME = "model"
BASE_MODEL_NAME = DIRECTORY_NAME + "/lraspp_mobilenet_v3_large"
weights_path = Path(BASE_MODEL_NAME + ".pt")

The torchvision module provides a ready to use set of functions for model class initialization. We will use torchvision.models.segmentation.lraspp_mobilenet_v3_large. You can directly pass pre-trained model weights to the model initialization function using weights enum LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1. However, for demonstration purposes, we will create it separately. Download the pre-trained weights and load the model. This may take some time if you have not downloaded the model before.

In [ ]:
print("Downloading the LRASPP MobileNetV3 model (if it has not been downloaded already)...") 
download_file(LRASPP_MobileNet_V3_Large_Weights.COCO_WITH_VOC_LABELS_V1.url, filename=weights_path.name, directory=weights_path.parent)
# create model object
model = lraspp_mobilenet_v3_large()
# read state dict, use map_location argument to avoid a situation where weights are saved in cuda (which may not be unavailable on the system)
state_dict = torch.load(weights_path, map_location='cpu')
# load state dict to model
model.load_state_dict(state_dict)
# switch model from training to inference mode
model.eval()
print("Loaded PyTorch LRASPP MobileNetV3 model")

## Convert the original model to OpenVINO IR Format

To convert the ONNX model to OpenVINO IR with `FP16` precision, use model conversion API. The models are saved inside the current directory. For more information on how to convert models, see this [page](https://docs.openvino.ai/2023.0/openvino_docs_model_processing_introduction.html).

In [ ]:
ov_model_xml_path = Path('models/ov_lraspp_model.xml')


if not ov_model_xml_path.exists():
    ov_model_xml_path.parent.mkdir(parents=True, exist_ok=True)
    dummy_input = torch.randn(1, 3, IMAGE_HEIGHT, IMAGE_WIDTH)
    # call tracing before converting. It is necessary in current version for segmentation models 
    traced_model = torch.jit.trace(model, dummy_input, strict=False)
    ov_model = ov.convert_model(traced_model, example_input=dummy_input)
    ov.save_model(ov_model, ov_model_xml_path)
else:
    print(f"IR model {ov_model_xml_path} already exists.")

## Show results
Confirm that the segmentation results look as expected by comparing model predictions on the OpenVINO IR and PyTorch models.

### Load and preprocess an input image

Images need to be normalized before propagating through the network.

In [ ]:
import cv2
import numpy as np


def normalize(image: np.ndarray) -> np.ndarray:
    """
    Normalize the image to the given mean and standard deviation
    for CityScapes models.
    """
    image = image.astype(np.float32)
    mean = (0.485, 0.456, 0.406)
    std = (0.229, 0.224, 0.225)
    image /= 255.0
    image -= mean
    image /= std
    return image

In [ ]:
from datasets import load_dataset


dataset = load_dataset("huggingface/cats-image")
image = dataset["test"]["image"][0]

display(image)

image = cv2.cvtColor(np.array(image), cv2.COLOR_BGR2RGB)

resized_image = cv2.resize(image, (IMAGE_WIDTH, IMAGE_HEIGHT))
normalized_image = normalize(resized_image)

# Convert the resized images to network input shape.
normalized_input_image = np.expand_dims(np.transpose(normalized_image, (2, 0, 1)), 0)

Select device from dropdown list for running inference using OpenVINO

In [ ]:
import ipywidgets as widgets

core = ov.Core()
device = widgets.Dropdown(
    options=core.available_devices + ["AUTO"],
    value='AUTO',
    description='Device:',
    disabled=False,
)

device

### Run an inference on the OpenVINO model

In [ ]:
compiled_model = core.compile_model(ov_model_xml_path, device_name=device.value)

In [ ]:
res_ir = compiled_model(normalized_input_image)[0]

Model predicts probabilities for how well each pixel corresponds to a specific label. To get the label with the highest probability for each pixel, operation argmax should be applied. After that, color coding can be applied to each label for more convenient visualization.

In [ ]:
from notebook_utils import segmentation_map_to_image, viz_result_image, SegmentationMap, Label


voc_labels = [
    Label(index=0, color=(0, 0, 0), name="background"),
    Label(index=1, color=(128, 0, 0), name="aeroplane"),
    Label(index=2, color=(0, 128, 0), name="bicycle"),
    Label(index=3, color=(128, 128, 0), name="bird"),
    Label(index=4, color=(0, 0, 128), name="boat"),
    Label(index=5, color=(128, 0, 128), name="bottle"),
    Label(index=6, color=(0, 128, 128), name="bus"),
    Label(index=7, color=(128, 128, 128), name="car"),
    Label(index=8, color=(64, 0, 0), name="cat"),
    Label(index=9, color=(192, 0, 0), name="chair"),
    Label(index=10, color=(64, 128, 0), name="cow"),
    Label(index=11, color=(192, 128, 0), name="dining table"),
    Label(index=12, color=(64, 0, 128), name="dog"),
    Label(index=13, color=(192, 0, 128), name="horse"),
    Label(index=14, color=(64, 128, 128), name="motorbike"),
    Label(index=15, color=(192, 128, 128), name="person"),
    Label(index=16, color=(0, 64, 0), name="potted plant"),
    Label(index=17, color=(128, 64, 0), name="sheep"),
    Label(index=18, color=(0, 192, 0), name="sofa"),
    Label(index=19, color=(128, 192, 0), name="train"),
    Label(index=20, color=(0, 64, 128), name="tv monitor")
]
VOCLabels = SegmentationMap(voc_labels)


In [ ]:
result_mask_ir = np.squeeze(np.argmax(res_ir, axis=1)).astype(np.uint8)
viz_result_image(
    image,
    segmentation_map_to_image(result=result_mask_ir, colormap=VOCLabels.get_colormap()),
    resize=True,
)

### Run an inference on the PyTorch model to compare results

In [ ]:
model.eval()
with torch.no_grad():
    result_torch = model(torch.as_tensor(normalized_input_image).float())

result_mask_torch = torch.argmax(result_torch['out'], dim=1).squeeze(0).numpy().astype(np.uint8)
viz_result_image(
    image,
    segmentation_map_to_image(result=result_mask_torch, colormap=VOCLabels.get_colormap()),
    resize=True,
)